In [ ]:
from google.colab import drive 
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
!pip install contractions

In [ ]:
from keras.models import load_model
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from textblob import TextBlob
import re
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
import contractions
import pickle
from numpy import *
from time import time

In [ ]:
# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 170000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 160
# This is fixed.
EMBEDDING_DIM = 100

In [ ]:
def clean_text(text):
  text=contractions.fix(text)
  text=re.sub(r'\W+',' ',text)
  text=re.sub('\s\s+',' ',text)
  text=text.strip()
  text=text.lower()
  return text

In [ ]:
train=pd.read_csv('gdrive/My Drive/CZ4034_train.csv')

In [ ]:
train['text']=train['text'].apply(clean_text)

In [ ]:
with open('gdrive/My Drive/tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

In [ ]:
CNN_LSTM = load_model("gdrive/My Drive/CZ4034_CNN_LSTM.h5")

In [ ]:
BI_LSTM = load_model("gdrive/My Drive/CZ4034_BI_LSTM.h5")

In [ ]:
LSTM = load_model("gdrive/My Drive/CZ4034_LSTM.h5")

In [ ]:
CNN = load_model("gdrive/My Drive/CZ4034_CNN.h5")

In [ ]:
def stacked_dataset(inputX):
    stackX = None

    # make prediction
    yhat1 = BI_LSTM.predict(pad_sequences(tokenizer.texts_to_sequences(inputX.values), maxlen=MAX_SEQUENCE_LENGTH), verbose=0)
    # stack predictions into [rows, members, probabilities]
    #yhat1=np.array([x for x in np.max(yhat1, axis=1)])
    #yhat1=yhat1.reshape()
    stackX = yhat1 #
    yhat2 = CNN_LSTM.predict(pad_sequences(tokenizer.texts_to_sequences(inputX.values), maxlen=MAX_SEQUENCE_LENGTH), verbose=0)
    #yhat2=np.array([x for x in np.max(yhat2, axis=1)])
    stackX = dstack((stackX, yhat2))
    #x = vec.fit_transform(inputX).toarray()
    yhat3 = LSTM.predict(pad_sequences(tokenizer.texts_to_sequences(inputX.values), maxlen=MAX_SEQUENCE_LENGTH), verbose=0)
    #yhat2=np.array([x for x in np.max(yhat2, axis=1)])
    stackX = dstack((stackX, yhat3))

    yhat4 = CNN.predict(pad_sequences(tokenizer.texts_to_sequences(inputX.values), maxlen=MAX_SEQUENCE_LENGTH), verbose=0)
    #yhat2=np.array([x for x in np.max(yhat2, axis=1)])
    stackX = dstack((stackX, yhat4))

    
    # flatten predictions to [rows, members x probabilities]
    stackX = stackX.reshape(stackX.shape[0],stackX.shape[1]*stackX.shape[2])
    return stackX

In [ ]:
from sklearn.linear_model import LogisticRegression


In [ ]:
def fit_stacked_model(inputX, inputy):
    # create dataset using ensemble
    stackedX = stacked_dataset( inputX)
    # fit the meta learner
    model = LogisticRegression() #meta learner
    model.fit(stackedX, inputy)
    return model

In [ ]:
# make a prediction with the stacked model
def stacked_prediction( model, inputX):
    # create dataset using ensemble
    stackedX = stacked_dataset( inputX)
    # make a prediction
    yhat = model.predict(stackedX)
    return yhat

In [ ]:
model=fit_stacked_model(train['text'],train['sentiment'])

In [ ]:
# saving
with open('gdrive/My Drive/Stacked Ensemble Model.pickle','wb') as handle:
    pickle.dump(model, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open('gdrive/My Drive/Stacked Ensemble Model.pickle','rb') as handle:
    model = pickle.load(handle)

In [ ]:
test=pd.read_csv('gdrive/My Drive/CZ4034_test.csv')

In [ ]:
test['text']=test['text'].apply(clean_text)

In [ ]:
start=time()
yhat1 = stacked_prediction(model,test['text'])
dur=time()-start
print(len(test)//dur)

274.0


In [ ]:
print(classification_report(test['sentiment'],yhat))

              precision    recall  f1-score   support

    negative       0.69      0.69      0.69       541
     neutral       0.81      0.79      0.80       647
    positive       0.79      0.81      0.80       769

    accuracy                           0.77      1957
   macro avg       0.76      0.76      0.76      1957
weighted avg       0.77      0.77      0.77      1957



In [ ]:
X_test = tokenizer.texts_to_sequences(test['text'].values)
X_test = pad_sequences(X_test, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X_test.shape)

Shape of data tensor: (1957, 160)


In [ ]:
start=time()
yhat2 = CNN.predict(X_test)
dur=time()-start
print(len(test)//dur)

785.0


In [ ]:
start=time()
yhat3 = LSTM.predict(X_test)
dur=time()-start
print(len(test)//dur)

1087.0


In [ ]:
start=time()
yhat4 = BI_LSTM.predict(X_test)
dur=time()-start
print(len(test)//dur)

295.0


In [ ]:
start=time()
yhat5 = CNN_LSTM.predict(X_test)
dur=time()-start
print(len(test)//dur)

1132.0
